In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from typing import NamedTuple
import itertools
import heapq
import functools


class PQ(object):
    def __init__(self):
        self.heap = []

    def push(self, item, priority):
        heapq.heappush(self.heap, (priority, item))
        
    def __len__(self):
        return len(self.heap)

    def pop(self):
        return heapq.heappop(self.heap)[1]
    
    def top(self):
        return self.heap[0][1]
    

@functools.total_ordering
class Car(object):
    def __init__(self, id_):
        self.id = id_
        self.current_street_idx = 0
        self.reached_at = -1
        self.score = 0
        self.streets = []
        
    def time_left(self):
        tl = 0
        if self.reached_at == -1:
            for street in self.streets[self.current_street_idx + 1:]:
                tl += 1 + street.length
        return tl
    
    def __hash__(self):
        return hash(self.id)
    
    def __eq__(self, other):
        return self.id == other.id
    
    def __le__(self, other):
        return self.id <= other.id

    
class Street(object):
    def __init__(self, id_):
        self.id = id_
        self.name = ''
        self.length = -1
        self.queue = PQ()
        self.use_count = 0
    
    def __hash__(self):
        return hash(self.id)
    
    def __eq__(self, other):
        return self.id == other.id


class Intersection(object):
    def __init__(self, id_):
        self.id = id_
        self.incoming_streets = set()
        self.outgoing_streets = set()
        
    def __hash__(self):
        return hash(self.id)
    
    def __eq__(self, other):
        return self.id == other.id


def read_input(f):
    street_name2o = {}
    duration, num_intersections, num_streets, num_cars, bonus = (int(x) for x in next(f).strip().split())
    intersections = [Intersection(i) for i in range(num_intersections)]
    streets = [Street(i) for i in range(num_streets)]
    cars = [Car(i) for i in range(num_cars)]
    
    for i in range(num_streets):
        bid, eid, name, length = next(f).strip().split()
        bid, eid, length = int(bid), int(eid), int(length)
        street = streets[i]
        street.name = name
        street.length = length
        street_name2o[street.name] = street
        
        intersections[eid].incoming_streets.add(street)
        intersections[bid].outgoing_streets.add(street)
    
    
    for i in range(num_cars):
        route = next(f).strip().split()
        for j, street_name in enumerate(route[1:]):
            street = street_name2o[street_name]
            cars[i].streets.append(street)
            street.use_count += 1
            if j == 0:
                street.queue.push(cars[i], 0) # time left to reach the end of street
                
    return duration, bonus, streets, intersections, cars
                

def dumb_strat_v3(streets, intersections, duration):
    """
    We will remove all unused streets from schedules and open each street for 1 second.
    Streets get priority in the schedule by (the top most waiting car's time to destination, queue size)
    Note this is a dumb strategy because we are not simulating anything
    """
    schedules = []
    for intersection in intersections:
        priority = []
        for street in intersection.incoming_streets:
            if len(street.queue):
                priority.append((street.queue.top().time_left(), len(street.queue), street.id, street.name))
            elif street.use_count:
                priority.append((duration + 1, 0, street.id, street.name))
        if priority:
            priority.sort(reverse=True)
            schedules.append(
                (intersection.id, [(street_name, 1) for _, _, _, street_name in priority])
            )
                
    return schedules

def dump_output(schedules, f):
    print(len(schedules), file=f)
    for intersection_id, street_pairs in schedules:
        print(intersection_id, file=f)
        print(len(street_pairs), file=f)
        for street_name, time in street_pairs:
            print(f'{street_name} {time}', file=f)

In [ ]:
def do():
    duration, bonus, streets, intersections, cars = read_input(open('/kaggle/input/hashcode-2021-oqr-extension/hashcode.in'))
    schedules = dumb_strat_v3(streets, intersections, duration)
    dump_output(schedules, open('submission.csv', 'w'))
                                                                    

do()

In [ ]:
! head -30 submission.csv